# 1 迁移学习
引用自：[keras系列︱Application中五款已训练模型、VGG16框架（Sequential式、Model式）解读（二）](http://www.cnblogs.com/Anita9002/p/8136535.html) 

Kera 的应用模块 `applications` 提供了带有预训练权重的 Keras 模型，这些模型可以用来进行预测、特征提取和 finetune。 
- Xception
- VGG16
- VGG19
- ResNet50
- InceptionV3
- Imagenet_utils
- Inception_ResNet_V2
- MobileNet

 ## [Writing your own Keras layers](https://keras.io/layers/writing-your-own-keras-layers/)

In [1]:
import keras

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Dense, Activation, Flatten, Input
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [6]:
from keras.layers import TimeDistributed

### 小技巧
#### 如何在keras中使用tensorboard
```python
RUN = RUN + 1 if 'RUN' in locals() else 1   # locals() 函数会以字典类型返回当前位置的全部局部变量。

    LOG_DIR = model_save_path + '/training_logs/run{}'.format(RUN)
    LOG_FILE_PATH = LOG_DIR + '/checkpoint-{epoch:02d}-{val_loss:.4f}.hdf5'   # 模型Log文件以及.h5模型文件存放地址

    tensorboard = TensorBoard(log_dir=LOG_DIR, write_images=True)
    checkpoint = ModelCheckpoint(filepath=LOG_FILE_PATH, monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

    history = model.fit_generator(generator=gen.generate(True), steps_per_epoch=int(gen.train_batches / 4),
                                  validation_data=gen.generate(False), validation_steps=int(gen.val_batches / 4),
                                  epochs=EPOCHS, verbose=1, callbacks=[tensorboard, checkpoint, early_stopping])
```

都是在回调函数中起作用：
- `EarlyStopping patience`：当 early 
  - stop被激活（如发现loss相比上一个epoch训练没有下降），则经过patience个epoch后停止训练。 
  - mode：‘auto’，‘min’，‘max’之一，在min模式下，如果检测值停止下降则中止训练。在max模式下，当检测值不再上升则停止训练。
- 模型检查点ModelCheckpoint  
  - save_best_only：当设置为True时，将只保存在验证集上性能最好的模型 
  -mode：‘auto’，‘min’，‘max’之一，在save_best_only=True时决定性能最佳模型的评判准则，例如，当监测值为val_acc时，模式应为max，当检测值为val_loss时，模式应为min。在auto模式下，评价准则由被监测值的名字自动推断。 
  - save_weights_only：若设置为True，则只保存模型权重，否则将保存整个模型（包括模型结构，配置信息等） 
  - period：CheckPoint之间的间隔的epoch数
- 可视化tensorboard write_images: 是否将模型权重以图片的形式可视化

In [9]:
import tensorflow as tf
import keras

AttributeError: module 'tensorflow.python.framework.tensor_util' has no attribute 'is_tensor'

In [7]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np

class MyLayer(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this somewhere!

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


AttributeError: module 'tensorflow.python.framework.tensor_util' has no attribute 'is_tensor'